In [ ]:
# Calcolo del punteggio TAG
def compute_tag_score(user_tags, answer_tags):
    intersection = set(user_tags.split('|')).intersection(set(answer_tags.split('|')))
    return len(intersection) / (len(user_tags.split('|')) + 1)

questions['user_tags'] = questions['Tags'].fillna("").apply(lambda x: x.split('|') if isinstance(x, str) else [])
answers['answer_tags'] = answers['Tags'].fillna("").apply(lambda x: x.split('|') if isinstance(x, str) else [])

questions['tag_score'] = questions.apply(
    lambda row: compute_tag_score(row['Tags'], answers.loc[row['AcceptedAnswerId']]['Tags']),
    axis=1
)
questions


In [ ]:
# Ottimizzazione dei pesi lambda
best_lambda = None
best_map = 0
for lambdas in [(0.1, 0.9), (0.2, 0.8), (0.3, 0.7)]:
    lambda_bm25, lambda_neural, lambda_tag = lambdas
    combined_scores = lambda_bm25 * retrieved['score'] + lambda_neural * retrieved['neural_score'] + lambda_tag * retrieved['tag_score']
    retrieved['combined_score'] = combined_scores
    map_score = pt.Utils.evaluate(retrieved, train_qrels, metrics=['map_100'])['map_100']
    if map_score > best_map:
        best_map = map_score
        best_lambda = lambdas

print("Best lambda:", best_lambda)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=144ec01f-394f-474f-b507-b786ab13b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>